In [1]:
# Scraping data from IMDB to use as the primary table
import pandas as pd
# To get timestamp
import time

import json

from requests import get
from bs4 import BeautifulSoup as bs 
url = "https://www.imdb.com/search/title/?genres=comedy&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=3396781f-d87f-4fac-8694-c56ce6f490fe&pf_rd_r=0GMQGWJ7C53HSZY7FNW4&pf_rd_s=center-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr1_i_1"
response = get(url)





html_soup = bs(response.text,'html.parser')
id_check = html_soup.find(id ="main")
movie_container = id_check.find_all(class_ ="lister-item mode-advanced")
container = movie_container[3]

movie_names = []
year_release = []
imdb_ratings = []
metascores = []
votes = []
movie_description = []
certificate = []
runtime = []
genre = []
director_name = []
star_cast = []
gross_value = []


for container in movie_container:

        #the movie_name
        name = container.h3.a.text
        movie_names.append(name)
        release = container.find("span", class_ = "lister-item-year text-muted unbold").text
        year_release.append(release)
        
        if container.strong is not None :
            ratings = float(container.strong.text)
            imdb_ratings.append(ratings)
        else:
            imdb_ratings.append(None)
        
        if container.find("div", class_ = "ratings-metascore")!=None:
            meta = container.find("span", class_ = "metascore").text
            metascores.append(int(meta))
        else:
            metascores.append(None)
        
        
        if  container.find("span", attrs = {"name":"nv"})!=None:
            vote = (container.find("span", attrs = {"name":"nv"}).text)
            vote = vote.split(",")
            votes.append(int("".join(vote)))
        else:
            votes.append(None)
            
            
        if container.find("span", class_ = "certificate")!=None:
            certificate.append(container.find("span", class_ = "certificate").text)
        else:
            certificate.append(None)
        
        #the runtime
        if container.find("span", class_ ="runtime")!= None:
            runtime.append(container.find("span", class_ ="runtime").text)
        else:
            runtime.append(None)
        
        gen = container.find("span", class_ ="genre").text
        genre.append(gen)
        
        #find  all <p> tags which contain the data
        content = container.find_all("p")
        
        #the description
        desc = content[1].text
        movie_description.append(desc)
        
        #subsetting all the <a> tags in 3rd <p> tag
        content_2 = content[2].find_all("a")

        #the director name
        if len(content_2)>3:
            director = content_2[0].text
        
        director_name.append(director)
        
        if len(container.find_all("span", attrs = {"name":"nv"})) >= 2:
            gross = container.find_all("span", attrs = {"name":"nv"})[1]['data-value']
            gross_value.append(gross)
        else:
            gross_value.append(None)
        #extracting artists names
        temp = []
        for i in range(len(content_2)-1):
            temp.append(content_2[i].text)
        star_cast.append(temp)

In [2]:

imdb_df = pd.DataFrame({"movie_names":movie_names,
                        "year_release":year_release,
                        "imdb_ratings":imdb_ratings,
                        "metascores":metascores,
                        "votes":votes,
                        "movie_description":movie_description,
                        "certificate":certificate,
                        "runtime":runtime,
                        "genre":genre,
                        "director_name": director_name,
                        "gross_value":gross_value
                       })

In [3]:
#Storing the IMDb movies in a data frame
imdb_df

,movie_names,year_release,imdb_ratings,metascores,votes,movie_description,certificate,runtime,genre,director_name,gross_value
0,Parasite,(2019),8.6,96.0,265908.0,"\n A poor family, the Kims, con their way i...",R,132 min,"\nComedy, Drama, Thriller",Bong Joon Ho,None
1,Jojo Rabbit,(2019),8.0,58.0,132181.0,\n A young boy in Hitler's army finds out h...,PG-13,108 min,"\nComedy, Drama, War",Taika Waititi,"349,555"
2,Sonic the Hedgehog,(2020),6.9,47.0,14401.0,"\n After discovering a small, blue, fast he...",PG,99 min,"\nAction, Adventure, Comedy",Jeff Fowler,None
3,Once Upon a Time... in Hollywood,(2019),7.7,83.0,390536.0,\n A faded television actor and his stunt d...,R,161 min,"\nComedy, Drama",Quentin Tarantino,"135,371,627"
4,Knives Out,(2019),8.0,82.0,183924.0,\n A detective investigates the death of a ...,PG-13,131 min,"\nComedy, Crime, Drama",Rian Johnson,None
5,The French Dispatch,(2020),NaN,NaN,NaN,\n A love letter to journalists set in an o...,R,108 min,"\nComedy, Drama, Romance",Wes Anderson,None
6,Sex Education,(2019– ),8.3,NaN,111281.0,\n A teenage boy with a sex therapist mothe...,TV-MA,45 min,"\nComedy, Drama",Asa Butterfield,None
7,Marriage Story,(2019),8.0,93.0,175580.0,\n Noah Baumbach's incisive and compassiona...,R,137 min,"\nComedy, Drama, Romance",Noah Baumbach,None
8,Fantasy Island,(2020),4.7,21.0,3455.0,\n A horror adaptation of the popular '70s ...,PG-13,109 min,"\nAdventure, Comedy, Horror",Jeff Wadlow,None
9,Dolittle,(2020),5.5,26.0,15295.0,\n A physician who can talk to animals emba...,PG,101 min,"\nAction, Adventure, Comedy",Stephen Gaghan,None


In [4]:
import tweepy

auth = tweepy.OAuthHandler('hzTTj9HU4Ru77Z7eKGO7IRXJw', 'WAKstXWAnWVvE2DYZciThLm4y9iuPVdoig4nFUMeQkLi5pU9rI')
auth.set_access_token('1231001224624738304-vbZOTSOt1a3ZhGFCsdmwkNtCMq8J78', 'TJ33XPyZNbVAaw7TqpNZFSxHUf6VNqvIBIcsbngKrXvrC')

api = tweepy.API(auth)
post=[]
tweet_id=[]
retweeted=[]
screenname=[]
users=["EllenPage","Scorsese4Ever","AndySamberg","jonnybernthal","ben_mckenzie","kj_apa","StephenAmell","ItsRamiMalek","BryanCranston","RealGaryRoss"]
timestamp=[]
name=[]

for username in users:
    
    for status in tweepy.Cursor(api.user_timeline, username).items(10):
        post.append(status.text)
        tweet_id.append(status.id)
        retweeted.append(status.retweeted)
        timestamp.append(status.created_at)
        screenname.append(status.user.screen_name)
        name.append(status.user.name)
        
dataframeMovie_1 = pd.DataFrame(
    {'post': post,
     'tweet_id': tweet_id,
     'retweeted': retweeted,
     'timestamp': timestamp,
     'screenname':screenname,
     'name':name
    })
print(dataframeMovie_1)

                                                 post             tweet_id  \
0   RT @DalhousieU: ICYMI: the documentary film "T...  1231379455765073923   
1   RT @DalNursing: .@EllenPage: “The more we film...  1231060163542626305   
2           @HABlackFilmFest @iwaldron2165 Thank you!  1231056482902462466   
3   RT @HABlackFilmFest: Happy Birthday, @EllenPag...  1231056411267936257   
4                               @etalkCTV Thank you 🙏  1230988627280322560   
..                                                ...                  ...   
86  Read Jimmy Kimmel’s emotional, scathing monolo...   915125195639185409   
87  The Smithsonian just put it out. Thanks to the...   914332622637088768   
88  Gary Ross, David Blight, Steven Hahn Honored t...   914331790122270720   
89  RT @dandrezner: I'm seriously beginning to thi...   909882631487029248   
90  Thought I'd put this up again https://t.co/gmu...   899025144055709696   

    retweeted           timestamp    screenname        name  
0

In [5]:
auth = tweepy.OAuthHandler('hzTTj9HU4Ru77Z7eKGO7IRXJw', 'WAKstXWAnWVvE2DYZciThLm4y9iuPVdoig4nFUMeQkLi5pU9rI')
auth.set_access_token('1231001224624738304-vbZOTSOt1a3ZhGFCsdmwkNtCMq8J78', 'TJ33XPyZNbVAaw7TqpNZFSxHUf6VNqvIBIcsbngKrXvrC')

api = tweepy.API(auth)
post=[]
tweet_id=[]
retweeted=[]
screenname=[]
users=["UmbrellaAcad","Irishman_Movie","nbcbrooklyn99","ThePunisher","Gotham","CW_Riverdale","CW_Arrow","whoismrrobot","BreakingBad","oceans8movie",]
timestamp=[]
name=[]

for username in users:
    
    for status in tweepy.Cursor(api.user_timeline, username).items(10):
        #print(status)
        post.append(status.text)
        tweet_id.append(status.id)
        retweeted.append(status.retweeted)
        timestamp.append(status.created_at)
        screenname.append(status.user.screen_name)
        name.append(status.user.name)
        
    
    

dataframeMovie_2 = pd.DataFrame(
    {'post': post,
     'tweet_id': tweet_id,
     'retweeted': retweeted,
     'timestamp': timestamp,
     'screenname':screenname,
     'name':name
    })
dataframeMovie_2



,post,tweet_id,retweeted,timestamp,screenname,name
0,@justinhmin rise n shine ☀️,1230219228445388800,False,2020-02-19 19:54:55,UmbrellaAcad,Umbrella Academy
1,@xalwayshigh @klauseyebrows @Beantacles @David...,1230196174549913601,False,2020-02-19 18:23:18,UmbrellaAcad,Umbrella Academy
2,@fangirlburnham stepping down from this positi...,1230196148775931906,False,2020-02-19 18:23:12,UmbrellaAcad,Umbrella Academy
3,@polarizespace @klauseyebrows @Beantacles @Dav...,1230196088948305922,False,2020-02-19 18:22:58,UmbrellaAcad,Umbrella Academy
4,@klauseyebrows @Beantacles @DavidCastanedaJ @j...,1230195736765177856,False,2020-02-19 18:21:34,UmbrellaAcad,Umbrella Academy
...,...,...,...,...,...,...
95,Wishing Helena Bonham Carter a happy birthday!...,1132692879116128257,False,2019-05-26 17:00:01,oceans8movie,Oceans8Movie
96,Wishing Cate Blanchett a very happy birthday! ...,1128344220262129664,False,2019-05-14 17:00:00,oceans8movie,Oceans8Movie
97,Which way to the Gala? https://t.co/nKwOPe3UfH,1125565016306925568,False,2019-05-07 00:56:26,oceans8movie,Oceans8Movie
98,"Happy Birthday Bad Gal, @rihanna. https://t.co...",1098275003378806784,False,2019-02-20 17:35:40,oceans8movie,Oceans8Movie


In [7]:
import sqlite3
conn = sqlite3.connect('TMDB_Database')


In [18]:
#Inserting the Company dataframe as company into the physical database
dataframeMovie_2.to_sql('users',con=conn,index=False, if_exists='replace')

#Inserting the Producer dataframe as producer into the physical database
dataframeMovie_1.to_sql('director',con=conn,index=False, if_exists='replace')

#Inserting the IMDb dataframe as consumer_2 into the physical database
imdb_df.to_sql('movies',con=conn,index=False, if_exists='replace')



In [15]:
dataframeMovie_2.to_csv('users.csv')
dataframeMovie_1.to_csv('director.csv')
imdb_df.to_csv('movies.csv')



In [10]:
def run_query(query):
    return pd.read_sql(query,conn)
run_query('select * from sqlite_master where type = "table"')

,type,name,tbl_name,rootpage,sql
0,table,company,company,2,"CREATE TABLE ""company"" (\n""post"" TEXT,\n ""twe..."
1,table,producer,producer,8,"CREATE TABLE ""producer"" (\n""post"" TEXT,\n ""tw..."
2,table,consumer_2,consumer_2,13,"CREATE TABLE ""consumer_2"" (\n""movie_names"" TEX..."


In [21]:
# Query 1: view all users
run_query('SELECT * from users')



,post,tweet_id,retweeted,timestamp,screenname,name
0,@justinhmin rise n shine ☀️,1230219228445388800,0,2020-02-19 19:54:55,UmbrellaAcad,Umbrella Academy
1,@xalwayshigh @klauseyebrows @Beantacles @David...,1230196174549913601,0,2020-02-19 18:23:18,UmbrellaAcad,Umbrella Academy
2,@fangirlburnham stepping down from this positi...,1230196148775931906,0,2020-02-19 18:23:12,UmbrellaAcad,Umbrella Academy
3,@polarizespace @klauseyebrows @Beantacles @Dav...,1230196088948305922,0,2020-02-19 18:22:58,UmbrellaAcad,Umbrella Academy
4,@klauseyebrows @Beantacles @DavidCastanedaJ @j...,1230195736765177856,0,2020-02-19 18:21:34,UmbrellaAcad,Umbrella Academy
...,...,...,...,...,...,...
95,Wishing Helena Bonham Carter a happy birthday!...,1132692879116128257,0,2019-05-26 17:00:01,oceans8movie,Oceans8Movie
96,Wishing Cate Blanchett a very happy birthday! ...,1128344220262129664,0,2019-05-14 17:00:00,oceans8movie,Oceans8Movie
97,Which way to the Gala? https://t.co/nKwOPe3UfH,1125565016306925568,0,2019-05-07 00:56:26,oceans8movie,Oceans8Movie
98,"Happy Birthday Bad Gal, @rihanna. https://t.co...",1098275003378806784,0,2019-02-20 17:35:40,oceans8movie,Oceans8Movie


In [29]:
# Query 2: When was this specific post created?
run_query('SELECT retweeted,timestamp,name FROM users WHERE screenname LIKE "%UmbrellaAcad%" AND tweet_id=1230219228445388800')



,retweeted,timestamp,name
0,0,2020-02-19 19:54:55,Umbrella Academy


In [35]:
#Query 3: Which are the movies with highest score?
run_query('SELECT * FROM movies WHERE metascores >=80  ORDER BY metascores DESC')

,movie_names,year_release,imdb_ratings,metascores,votes,movie_description,certificate,runtime,genre,director_name,gross_value
0,Parasite,(2019),8.6,96.0,265908.0,"\n A poor family, the Kims, con their way i...",R,132 min,"\nComedy, Drama, Thriller",Bong Joon Ho,None
1,Marriage Story,(2019),8.0,93.0,175580.0,\n Noah Baumbach's incisive and compassiona...,R,137 min,"\nComedy, Drama, Romance",Noah Baumbach,None
2,Toy Story 4,(2019),7.9,84.0,158051.0,"\n When a new toy called ""Forky"" joins Wood...",G,100 min,"\nAnimation, Adventure, Comedy",Josh Cooley,"433,033,071"
3,Booksmart,(2019),7.2,84.0,65717.0,\n On the eve of their high school graduati...,R,102 min,\nComedy,Olivia Wilde,"22,680,962"
4,Once Upon a Time... in Hollywood,(2019),7.7,83.0,390536.0,\n A faded television actor and his stunt d...,R,161 min,"\nComedy, Drama",Quentin Tarantino,"135,371,627"
5,Knives Out,(2019),8.0,82.0,183924.0,\n A detective investigates the death of a ...,PG-13,131 min,"\nComedy, Crime, Drama",Rian Johnson,None


In [40]:
# Query 4: What are the most recent tweets from this user?

run_query('SELECT * FROM users WHERE screenname = "oceans8movie" and timestamp > "2020-02-20 00:00:00" AND timestamp < "2020-02-22 23:59:59"')


,post,tweet_id,retweeted,timestamp,screenname,name
0,"Happy Birthday to our brilliant Nine Ball, Rih...",1230560516159889410,0,2020-02-20 18:31:04,oceans8movie,Oceans8Movie


In [43]:
#Query 5: Which movie has most votes:
run_query('SELECT votes,movie_names,year_release,imdb_ratings  FROM movies ORDER BY votes DESC')

,votes,movie_names,year_release,imdb_ratings
0,737902.0,Friends,(1994–2004),8.9
1,679602.0,The Big Bang Theory,(2007–2019),8.1
2,534286.0,Thor: Ragnarok,(2017),7.9
3,390536.0,Once Upon a Time... in Hollywood,(2019),7.7
4,342543.0,Suits,(2011–2019),8.5
5,338808.0,The Office,(2005–2013),8.8
6,322827.0,Modern Family,(2009– ),8.4
7,321552.0,Rick and Morty,(2013– ),9.3
8,292027.0,Green Book,(2018),8.2
9,265908.0,Parasite,(2019),8.6


In [45]:
#Query 6: Who directed this movie?
run_query('SELECT director_name FROM movies WHERE movie_names LIKE "%Hedgehog"')

,director_name
0,Jeff Fowler


,post,tweet_id,retweeted,timestamp,screenname,name
0,"RT @DalhousieU: ICYMI: the documentary film ""T...",1231379455765073923,0,2020-02-23 00:45:14,EllenPage,Ellen Page
1,RT @DalNursing: .@EllenPage: “The more we film...,1231060163542626305,0,2020-02-22 03:36:29,EllenPage,Ellen Page
2,@HABlackFilmFest @iwaldron2165 Thank you!,1231056482902462466,0,2020-02-22 03:21:52,EllenPage,Ellen Page
3,"RT @HABlackFilmFest: Happy Birthday, @EllenPag...",1231056411267936257,0,2020-02-22 03:21:35,EllenPage,Ellen Page
4,@etalkCTV Thank you 🙏,1230988627280322560,0,2020-02-21 22:52:14,EllenPage,Ellen Page
...,...,...,...,...,...,...
86,"Read Jimmy Kimmel’s emotional, scathing monolo...",915125195639185409,0,2017-10-03 08:03:43,RealGaryRoss,Gary Ross
87,The Smithsonian just put it out. Thanks to the...,914332622637088768,0,2017-10-01 03:34:19,RealGaryRoss,Gary Ross
88,"Gary Ross, David Blight, Steven Hahn Honored t...",914331790122270720,0,2017-10-01 03:31:00,RealGaryRoss,Gary Ross
89,RT @dandrezner: I'm seriously beginning to thi...,909882631487029248,0,2017-09-18 20:51:38,RealGaryRoss,Gary Ross
